In [338]:
import pandas as pd
import numpy as np

In [339]:
df=pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [340]:
df.isna().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [341]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [342]:
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].fillna('NA')

In [343]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [344]:
numeric_columns = list(df.dtypes[df.dtypes != 'object'].index)

for col in numeric_columns:
    df[col] = df[col].fillna(0)

In [345]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [346]:
df.industry.mode()


0    retail
Name: industry, dtype: object

In [347]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count','lead_score']


In [348]:
df_full_train['interaction_count'].corr(df_full_train['lead_score'])

np.float64(0.025393144769178252)

In [349]:
df_full_train['annual_income'].corr(df_full_train['interaction_count'])

np.float64(0.011958845353605895)

In [350]:
df_full_train['number_of_courses_viewed'].corr(df_full_train['lead_score'])

np.float64(0.009427127122612573)

In [351]:
df_full_train['number_of_courses_viewed'].corr(df_full_train['interaction_count'])

np.float64(-0.04438081449493219)

In [352]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score

In [353]:
df.shape

(1462, 9)

In [354]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [355]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [356]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [357]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [358]:
def mutual_info_converted(series):
    return mutual_info_score(series, df_full_train.converted)

In [359]:
df_full_train[categorical].head

<bound method NDFrame.head of        lead_source       industry employment_status       location
1066  social_media  manufacturing     self_employed      australia
638         events         retail           student  north_america
799   social_media      education                NA         europe
380       referral      education          employed      australia
303       paid_ads     healthcare          employed         europe
...            ...            ...               ...            ...
1130        events  manufacturing          employed  south_america
1294        events     healthcare           student  south_america
860       paid_ads  manufacturing           student  north_america
1459      paid_ads     technology           student  north_america
1126        events  manufacturing     self_employed           asia

[1169 rows x 4 columns]>

In [360]:
mi = df_full_train[categorical].apply(mutual_info_converted)
mi.sort_values(ascending=False).round(2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

In [361]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [362]:
df_train.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
dtype: int64

In [363]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [364]:
X_train.shape

(876, 31)

In [365]:
X_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))

In [366]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [367]:
y_pred = model.predict_proba(X_val)[:, 1]

In [368]:
converted = (y_pred >= 0.5)

In [369]:
(y_val == converted).mean().round(2)

np.float64(0.7)

In [370]:
baseline_acc = (y_val == converted).mean()
baseline_acc

np.float64(0.6996587030716723)

In [371]:
features_to_test = ['industry', 'employment_status', 'lead_score']
results = {}

print("Baseline accuracy (all features):", baseline_acc)


for feature in features_to_test:
    print(f"\nDropping feature: {feature}")
    dropped_categorical = [f for f in categorical if f != feature]
    dropped_numerical = [f for f in numerical if f != feature]
    dropped_features = dropped_categorical + dropped_numerical

    dv = DictVectorizer(sparse=False)
    train_dict = df_train[dropped_features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[dropped_features].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    converted = (y_pred >= 0.5)
    acc = (y_val == converted).mean()
    diff = baseline_acc - acc

    results[feature] = diff
    print(f"Accuracy (without {feature}): {acc}")
    print(f"Accuracy difference: {diff}")

print("\nSummary of accuracy differences:")
for f, d in results.items():
    print(f"{f}: {d.round(4)} (lower is less important)")

Baseline accuracy (all features): 0.6996587030716723

Dropping feature: industry
Accuracy (without industry): 0.6996587030716723
Accuracy difference: 0.0

Dropping feature: employment_status
Accuracy (without employment_status): 0.6962457337883959
Accuracy difference: 0.0034129692832763903

Dropping feature: lead_score
Accuracy (without lead_score): 0.7064846416382252
Accuracy difference: -0.0068259385665528916

Summary of accuracy differences:
industry: 0.0 (lower is less important)
employment_status: 0.0034 (lower is less important)
lead_score: -0.0068 (lower is less important)


In [373]:
C_values = [0.01, 0.1, 1, 10, 100]
results = {}


dv = DictVectorizer(sparse=False)
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)


for c in C_values:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    converted = (y_pred >= 0.5)
    acc = (y_val == converted).mean()
    results[c] = round(acc, 3)
    print(f"C={c}: Accuracy={round(acc, 3)}")


best_acc = max(results.values())
best_Cs = [c for c in C_values if results[c] == best_acc]
best_C = min(best_Cs)

print(f"\nBest accuracy: {best_acc}")
print(f"Best C (smallest if multiple): {best_C}")


C=0.01: Accuracy=0.7
C=0.1: Accuracy=0.7
C=1: Accuracy=0.7
C=10: Accuracy=0.7
C=100: Accuracy=0.7

Best accuracy: 0.7
Best C (smallest if multiple): 0.01
